<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Final-Project-Check-in" data-toc-modified-id="Final-Project-Check-in-1">Final Project Check-in</a></span></li><li><span><a href="#Group-Name" data-toc-modified-id="Group-Name-2">Group Name</a></span></li><li><span><a href="#Student-Names" data-toc-modified-id="Student-Names-3">Student Names</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-4">Load Data</a></span></li><li><span><a href="#Fit-scikit-learn-model" data-toc-modified-id="Fit-scikit-learn-model-5">Fit scikit-learn model</a></span></li><li><span><a href="#Evaluation-Metric" data-toc-modified-id="Evaluation-Metric-6">Evaluation Metric</a></span></li></ul></div>

Final Project Check-in
------

Group Name: SAL
-----

Student Names
----

1. Lisa Chua
2. Alaa A Latif
3. Shane Buchanan 

In [17]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [74]:
df = df.replace('unknown', np.nan)

In [25]:
def clean_data(df):
    df = df.replace('unknown', np.nan)
    return df

Load Data
-----

In [42]:
reset -fs

In [65]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn import compose
from sklearn.experimental import enable_iterative_imputer
from sklearn import impute
from sklearn.linear_model import LogisticRegression


In [ ]:
# Create our imputer to replace missing values with the median
SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [3]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('~/data/bank-additional/bank-additional-full.csv', delimiter=';')

In [141]:
# Now, upsample the fraud records in training set
df_no = df[df['y']=='no']
df_yes = df[df['y']=='yes']

df_yes_upsampled = df_yes.sample(int(len(df_yes)*10), replace=True)
df_upsampled = pd.concat([df_no, df_yes_upsampled], axis=0)

In [142]:
def make_pipeline(classifier=None):
    "Create a single pipeline that processes the data and then fits the regressor." 
    
    # YOUR CODE HERE
    numeric_features     = ['age', 'campaign', 'cons.price.idx']
    categorical_features = ['job', 'education', 'default', 'housing',
                            'loan', 'campaign', 'poutcome']
    
    categorical_transformer = Pipeline(steps=[
    ('impute', impute.SimpleImputer(strategy='most_frequent', add_indicator=True)),
    ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))])
    
    numeric_transformer = Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='median')),
    ('scaler', preprocessing.StandardScaler())])
    
    
    
    preprocessor = compose.ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])
    
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                       ('classifier', classifier)])
    return pipeline

In [143]:
lb = preprocessing.LabelBinarizer()

In [155]:
y = lb.fit_transform(df_upsampled['y'])

In [156]:
# Features data.
x = df_upsampled.drop('y', axis=1)

In [158]:
y = df_upsampled['y']

In [165]:
lb = preprocessing.LabelBinarizer()
y = lb.fit_transform(df['y'])
x = df.drop('y', axis=1)
y = df['y']

In [166]:
# Train-test-split on data.
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

Fit scikit-learn model
----

In [160]:
# Initialize dtree and fit.
clf = DecisionTreeClassifier()
pipeline = make_pipeline(clf)

## Predict

In [161]:
# Train Estimator
pipeline.fit(x_train, y_train);

In [167]:
y_pred = pipeline.predict(x_val)

Evaluation Metric
----

In [168]:
# Use F1 score as metric since we have an unbalanced data set.
f1_score(y_val, y_pred, average='weighted')

0.9488782068594529

In [164]:
# Print confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_val, y_pred)
print(cm)

[[6428  972]
 [  38 9152]]
